# Importing Libararies


In [167]:
from sympy import *
import random
import sys
import numpy as np
import itertools
from scipy.stats import expon

# Input of the algorithms

In [168]:
#input = " (~a) & (b | a) & (~c | ~b) & (~c) & (~c | d) & (~e | ~a )" ## answer is 4, d = 2, e = 2
input = " (~a) & (~c | ~b) & (~c) & (c | d) & (~e | ~a )" ## answer is 4, d = 2, e = 2, b = 2
#input =" (~a) & (b | a) & (~c | ~b) & (~c) & (~c | d) & (~e | ~a) & (~f) & (~f | g)" ## answer is 8, d = 2, e = 2, g = 2
#input = " (~a) & (b | a) & (c | ~b) & (c) & (~c | d) & (e | ~a) & (f) & (~f | g) " ## The answer is 2
#input = " (~a) & (b | a) & (c | ~b) & (c) & (~c | d) & (e | a) & (f) & (~f | g) " ## The answer is 1

# Pre-processing of the input

In [169]:
# Preprocessing level for recognizing clauses in formulas of CNForm and literals as variables

def preprocess(input):
  """ 
   The output is a tuple with three returns inside the first one a list of literals, the second one the tuple of cluases and
   third one is the formula itself sympified
  """
  input = sympify(input, evaluate=False)
  literals = input.atoms()
  clause = clauses(input)
  return literals, clause, input 
## for CNFs only
def clauses(input): 
  input = sympify(input, evaluate=False)
  if not isinstance(input, And):
      return input,
  return sympify(input.args)

print(clauses(input))
print(type(clauses(input)))
print(preprocess(input)[0])
print(type(preprocess(input)[0]))
print(type(preprocess(input)[1]))
print(type(preprocess(input)[2]))

(~a, ~c, c | d, ~a | ~e, ~b | ~c)
<class 'sympy.core.containers.Tuple'>
{d, e, b, c, a}
<class 'set'>
<class 'sympy.core.containers.Tuple'>
And


# utilities

In [170]:
def decision(p = 0.5):
    """
    returns a bool variable with probability of p being true
    """
    return random.random() < p 

def randomDict(literals):
    """
    Randomly assinginig true and false to literals of the input formula 
    returns dictionary 
    """
    baseDict = {}
    for atom in literals:   # Randomly initilizing the self.literals to T and F
        baseDict[atom] = random.choice([True, False])
    return baseDict

def unitprop(input, literal):
  vars = list(preprocess(input)[0])
  output = []
  for var in vars:
    if literal != var:
      output.append(var)  
  print("output:", output)
  return output
  
#vars = list(preprocess(input)[0])
#print("vars: ", vars)
#print("lit: ", lit)
#unitprop(input, lit)

# ChooseVar Algorithm 

In [171]:
#"  (~a | b) & (~b | ~c) & (~c) & (~a) "

def ChooseVar(sT, clF, dict1):
    clF = clF
    cho1 = random.choice(list(clF))
    sT = sT
    dict2 = dict1
    sT1 = set()
    dict3 = dict2.copy()
    dict4 = dict2.copy()
    bvl = []
    vars = list(preprocess(input)[0])
    clauses1 = clauses(sT)
    clauses1 = set(clauses1)
    sat = []
    clF =  set() 
    vltf = str()
    clT = clauses1.copy()
    clT = set(clT)
    ##### Break value calculation
    for lit in preprocess(cho1)[0]:
        dict2[lit] = not dict2[lit]
        dict1 = dict2
        # The set that we put the clauses that are still false inside
        for clause in clauses1:   
            temp = clause
            for key in dict1.keys():
                if key in clause.atoms():
                    clause = clause.subs({key: dict1[key]})
                # Adding false clauses to the set
                if not clause:
                    clF.add(temp)
                if temp in clF:
                    clT.remove(temp)
        bv = len(sT1) - len(clT)
        bvl.append(max(0, bv))
        dict3.update({lit: [not dict2[lit], max(0, bv)]})
    # chooseVar 
    vltflst = []      # value to flip list 
    probability= 0.5  ##### q probability  
    for key, value in dict3.items():
        if not type(value) == bool :
            if value[1] == 0 :          # if break value equal 0 chose the variable
                vltflst.extend([key, value[1]])
                vltf = key              # value to flip 
        else :
            if decision(probability):               # with probability of q chose randomly between the vars in cho1
                vltf = random.sample(preprocess(cho1)[0], 1)[0]
            else:                                   # with probability 1-q chose the min of break values in among the literals
                try:
                    vltf = vltflst[vltflst.index(min(vltflst))-1]      
                except ValueError:
                    pass
                except TypeError:
                    pass
    return vltf

# Metropolis Algorithm

In [172]:

def Metropolis(sT, dict1, T = 1):
    t = T
    sT = sT
    vars = list(preprocess(input)[0])
    dict1 = dict1
    costl = []
    #if len(dict1) == 0:
    # Randomly initilizing the vars to T and F
    dict1 = randomDict(vars)
    #dict1[atom]= True  # for testing
    dict2 = dict1.copy()
    dict3 = dict2.copy()
  #==============================================================================================================
  ##### Cost calculation
    clauses1 = clauses(sT)
    clauses1 = set(clauses1)
    sat = []
    clF =  set() 
    clT = clauses1.copy()
    vltf = str()
    clauses1 = clauses(input)
    clauses1 = set(clauses1)
    clf =  set() 
    for lit in vars:
        dict2[lit] = not dict2[lit]
        dict1 = dict2    
        dict1 = dict2.copy()
        clt = clauses1.copy()
        for clause in clauses1:   
            temp = clause
            #print("clause before: ", clause)
            for key in dict1.keys():
                if key in clause.atoms():
                    clause = clause.subs({key: dict1[key]})
            # Adding false clauses to the set
                if not clause:
                    clf.add(temp)
            if temp in clf:
                clt.remove(temp)
        sT1 = clt
        bv = len(sT) - len(sT1)
        costl.append(bv)
        #print("costl: ", costl)
        dict3.update({lit: [not dict2[lit], bv]}) 
    #print("dict3: ",dict3)
    ######==========================================================
    vltf = random.choice(vars)
    costVltf = dict3[vltf][1]
    #print("costVltf: ", costVltf)
    ex = expon.pdf(costVltf) / t
    #print("expon.pdf(costVltf) / t: ", ex)
    rand = random.random()
    #print("rand: ", rand)
    if costVltf <= 0:
        return [vltf, True]
    elif rand  < ex:
        return [vltf , True]
    else:
        return [vltf , not vltf]





# Sample Sat algorithm
This algorithm combines random walk and metropolis moves. 

In [173]:
def PRW(input, P = 0.5, T = 1):  # Returns the set of false clauses, set of truth clauses, the clause chosen at random from False cluases
                  # Also returns formula is satisfied when there is no false clauses and the dict of assigned values to vars
  p = P
  t = T
  vars = list(preprocess(input)[0])
  clauses1 = clauses(input)
  dict1 = {}
  isat = 0
  sat = []
  dict1 = randomDict(vars)
      #dict1[atom]= True  # for testing
  #print("clauses1: ",clauses1)
  clauses1 = list(clauses1)
  clF =  set()     # The set that we put the clauses that are still false inside
  clT = clauses1.copy()
  vltf = str()
  i0 =0
  while i0 in range(0,1000):
    #print("i0 : ", i0)
    #print("dict1: ", dict1)
    i0 += 1
    if len(clF) != 0:
      if vltf[0] in dict1 & vltf[1]:
        dict1[vltf[0]] = not dict1[vltf[0]]
    for clause in clauses1:   
      temp = clause
      #print("clause before: ", clause)
      for key in dict1.keys():
        clT = set(clT)
        if key in clause.atoms():
          clause = clause.subs({key: dict1[key]})
          #print("clause after: ", clause)              # Adding false clauses to the set
        if not clause:
          clF.add(temp)
      if temp in clF:
        clT.remove(temp)
    #print("clF: ", clF)
    #print("clT: ", clT)
    if len(clF) == 0:
      sat.append(dict1)
      #print("dict1: ", dict1)
      #print("The formula is satisfied") 
      break
    if len(clF) != 0:
      if decision(p):
        vltf = ChooseVar(clT, clF, dict1)
        #print("We are using chooseVar")
      else: 
        vltf = Metropolis(clT, dict1 , T = t)
        #print("We are using Metropolis")
      clF = set()      # The set that we put the clauses that are still false inside
      clT = clauses1.copy()
  return clF, clT, 0, dict1, sat
     


#prw = PRW(input)
#print("PRW: ",prw)
#print("prw[1] :",prw[1])
#print(PRW(prw[1],prw[3]))

# ApproxCount Algorithm

In [178]:
## ApproxCount Algorithm
k = 20
nsat = []
nsatSet = []
inputSet = preprocess(input)
#==============
# constructiong solution space of
for i in range(0,100):   # 
    anw = PRW(input)[3]
    anw1 = anw.copy()
    #print("anw: ", anw1)
    #anw = prw[4]
    nsat.append(anw1)
    #print("nsat: ", nsat)
    ##  Dropping duplicates in the nsat list 
    if anw1 not in nsatSet:   
        nsatSet.append(anw1)
    if len(nsatSet) == k:
        break 

print("nsat: ", nsat)
print("nsatSet: ", nsatSet, len(nsatSet))
print("nsatSet: length: ", len(nsatSet))
k = len(nsatSet)
mx = []
pickvarF = list(inputSet[0])
print("pickvarF: ", pickvarF)

for x in pickvarF:
    print("x: ", x)
    nf = 0.0
    nt = 0.0
    for sat in nsatSet:
        print("sat: ", sat)
        dic = sat
        x1 = dic[x]
        print("x1: ", x1)
        if x1 == True:
            nt += 1.0  
        if x1 == False:
            nf += 1.0 
    if nt > nf:
        ## unitprop F
        #pickvarF = unitprop(pickvarF, x)
        print("nt: ", nt)
        temp = k/nt
        mx.append(temp)
        print("mxt:", mx)
    else:
        ### unitprop F
        #pickvarF = unitprop(pickvarF, x)
        temp = k/nf
        mx.append(temp)
        print("mxf:", mx)
print("pickvarF: ", pickvarF)
print("mx:", mx)
result = np.prod(mx) 
print("result:", result)

nsat:  [{d: True, e: False, b: True, c: False, a: False}, {d: True, e: False, b: True, c: False, a: False}, {d: True, e: False, b: True, c: False, a: False}, {d: True, e: False, b: False, c: False, a: False}, {d: True, e: False, b: True, c: False, a: False}, {d: True, e: False, b: False, c: False, a: False}, {d: True, e: False, b: False, c: False, a: False}, {d: True, e: False, b: False, c: False, a: False}, {d: True, e: False, b: False, c: False, a: False}, {d: True, e: False, b: True, c: False, a: False}, {d: True, e: True, b: False, c: False, a: False}, {d: True, e: True, b: False, c: False, a: False}, {d: True, e: False, b: False, c: False, a: False}, {d: True, e: False, b: False, c: False, a: False}, {d: True, e: False, b: False, c: False, a: False}, {d: True, e: False, b: True, c: False, a: False}, {d: True, e: True, b: False, c: False, a: False}, {d: True, e: True, b: False, c: False, a: False}, {d: True, e: False, b: False, c: False, a: False}, {d: True, e: True, b: False, c: F

list of websites that I used: 

https://docs.sympy.org/latest/modules/logic.html

https://docs.sympy.org/latest/modules/logic.html#simplification-and-equivalence-testing

https://docs.sympy.org/latest/modules/parsing.html

https://docs.sympy.org/latest/modules/core.html

https://docs.python.org/3.7/tutorial/errors.html#tut-handling

https://www.w3schools.com/python/python_howto_remove_duplicates.asp
